# Aggression Behavior Implementation

**Step 1: Create display widgets**

In [ ]:
import ipywidgets.widgets as widgets
from IPython.display import display
import cv2

# Create widgets for displaying images
display_color = widgets.Image(format='jpeg', width='30%')
display_depth = widgets.Image(format='jpeg', width='30%')
layout = widgets.Layout(width='100%')

sidebyside = widgets.HBox([display_color, display_depth], layout=layout)
display(sidebyside)

# Convert numpy array to jpeg coded data for displaying
def bgr8_to_jpeg(value):
    return bytes(cv2.imencode('.jpg', value)[1])

**Step 2: Initialize camera and implement aggression behavior**

The robot will exhibit aggression behavior by:
- Approaching objects when they are far away (> 1000mm)
- Robot will rush or apporach you will speed

This creates an attraction effect where the robot seeks to stay near objects at a comfortable distance.

In [ ]:
import numpy as np
import pyzed.sl as sl
import threading
import motors

# Initialize the Robot class
robot = motors.MotorsYukon(mecanum=False)

class Camera():
    def __init__(self):
        super(Camera, self).__init__()

        self.zed = sl.Camera()
        # Create a InitParameters object and set configuration parameters
        init_params = sl.InitParameters()
        init_params.camera_resolution = sl.RESOLUTION.VGA
        init_params.depth_mode = sl.DEPTH_MODE.ULTRA
        init_params.coordinate_units = sl.UNIT.MILLIMETER

        # Open the camera
        status = self.zed.open(init_params)
        if status != sl.ERROR_CODE.SUCCESS:
            print("Camera Open : "+repr(status)+". Exit program.")
            self.zed.close()
            exit(1)

        # Create and set RuntimeParameters after opening the camera
        self.runtime = sl.RuntimeParameters()

        # Flag to control the thread
        self.thread_runnning_flag = False

        # Get the height and width
        camera_info = self.zed.get_camera_information()
        self.width = camera_info.camera_configuration.resolution.width
        self.height = camera_info.camera_configuration.resolution.height
        self.image = sl.Mat(self.width, self.height, sl.MAT_TYPE.U8_C4, sl.MEM.CPU)
        self.depth = sl.Mat(self.width, self.height, sl.MAT_TYPE.F32_C1, sl.MEM.CPU)
        self.point_cloud = sl.Mat(self.width, self.height, sl.MAT_TYPE.F32_C4, sl.MEM.CPU)

    def _capture_frames(self):
        while(self.thread_runnning_flag == True):
            if self.zed.grab(self.runtime) == sl.ERROR_CODE.SUCCESS:
                
                # Retrieve Left image
                self.zed.retrieve_image(self.image, sl.VIEW.LEFT)
                # Retrieve depth map. Depth is aligned on the left image
                self.zed.retrieve_measure(self.depth, sl.MEASURE.DEPTH)
    
                self.color_value = self.image.get_data()
                # Scale for real-time data displaying
                scale = 0.1
                resized_image = cv2.resize(self.color_value, None, fx=scale, fy=scale, 
                                          interpolation=cv2.INTER_AREA)
                cv2.circle(resized_image, (int(self.width*scale//2), int(self.height*scale//2)), 
                          1, (0, 255, 0))
                display_color.value = bgr8_to_jpeg(resized_image)
                
                self.depth_image = np.asanyarray(self.depth.get_data())

                # Process depth image for aggression behavior
                depth_image_test = self.depth_image.copy()
                depth_image_test = np.nan_to_num(depth_image_test, nan=0.0).astype(np.float32)
                
                # Define central area (similar to Tutorial 3 Part B)
                depth_image_test[:94, :] = 0
                depth_image_test[282:, :] = 0
                depth_image_test[:, :168] = 0
                depth_image_test[:, 504:] = 0

                # Filter depth values
                depth_image_test[depth_image_test < 100] = 0
                depth_image_test[depth_image_test > 2000] = 0
                
                depth_colormap = cv2.applyColorMap(
                    cv2.convertScaleAbs(depth_image_test, alpha=0.03), 
                    cv2.COLORMAP_JET)
        
                # Aggression Behaviour start
                if(depth_image_test.max() == 0):
                    #checking for people
                    robot.spin(0.3)
                    cv2.putText(depth_colormap, 'Checking..', (250, 188), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.6, (128, 128, 128), 2, cv2.LINE_AA)
                    print('Checking for target', end='\r')
                else:
                    # Found target
                    distance = depth_image_test[depth_image_test != 0].min()

                    # Bull movement to the target
                    if distance > 1000:
                        # bull movement: the target is far go fast at high speed
                        robot.forward(0.8)
                        cv2.putText(depth_colormap, f'Rushing {distance:.0f}mm', (220, 188),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 225), 2, cv2.LINE_AA)
                        print(f'Bull Attart {distance:.0f}mm', end='\r')
                    elif distance > 500:
                    # slown down when closer to target
                        robot.forward(0.4)
                        cv2.putText(depth_colormap, f'Getting closer {distance:.0f}mm', (220, 188),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 165, 225), 2, cv2.LINE_AA)
                        print(f'Closing in {distance:.0f}mm', end='\r')
                    else:  # distance < 500
                    # will stop when close to target
                        robot.stop()
                        cv2.putText(depth_colormap, f'confronting {distance:.0f}mm', (240, 188),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 255), 2, cv2.LINE_AA)
                        print(f'Bull will stop {distance:.0f}mm', end='\r')   
                resized_depth_colormap = cv2.resize(depth_colormap, None, fx=scale, fy=scale,
                                                    interpolation=cv2.INTER_AREA)
                display_depth.value = bgr8_to_jpeg(resized_depth_colormap)
    
    def start(self):
        if self.thread_runnning_flag == False:
            self.thread_runnning_flag = True
            self.thread = threading.Thread(target=self._capture_frames)
            self.thread.start()

    def stop(self):
        if self.thread_runnning_flag == True:
            self.thread_runnning_flag = False
            self.thread.join()
            robot.stop()

camera = Camera()
camera.start()